In [1]:
import tweepy as tw, sys, pandas as pd
import json
import csv
from datetime import date
from datetime import datetime
import time
import os
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from pathlib import Path
import numpy as np 
from datetime import datetime,date

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ddevii/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:

# your Twitter API key and API secret
my_api_key = "RXZTjQPqTznsj2wLewg2NlN2j"
my_api_secret = "xFOywT5of6TzQvu0aemBX5MBVmrvfKdlqzYRvn9q6zzYHbVExT"
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
search_query = "bitcoin -filter:retweets"

In [13]:

# get tweets from the API
tweets = tw.Cursor(api.search_tweets,
              q=search_query,
              lang="en",
              until = "2022-07-07"
              ).items(20)
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 20


In [14]:
# initialize the dataframe
tweets_df = pd.DataFrame()
tweets_df

""


In [17]:
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': "date",
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

In [18]:
# show the dataframe
tweets_df

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,💖 sailor symphony 💖,cathy's mom's house,24 - she/they - renowned talks-too-mucher - do...,False,<class 'datetime.date'>,involuntarily own bitcoin now,None,Twitter for iPhone
1,Crypto Watch Bot,"New York, NY",I crunch cryptocurrency data to find the best ...,False,<class 'datetime.date'>,#Investing 85.4% in this optimal #crypto portf...,"[Investing, crypto, cash]",Crypto Watch Bot
2,Wallat Capital,"Florida, USA",DM for telegram info...https://t.co/4sXvKDs1b8,False,<class 'datetime.date'>,🔥Bitcoin 100 to however much challenge Live!\n...,None,Twitter Web App
3,Crapto,,I battle frogs. 0% advice 100% crap \n\n@matts...,False,<class 'datetime.date'>,@BJusty1 Interesting point. Yeah nothing compa...,None,Twitter Web App
4,Sir Darwin,,Biologist. Non-toxic maximalist. \n\n#Bitcoin\...,False,<class 'datetime.date'>,What I do is think of bitcoin as a long-term s...,None,Twitter Web App
5,PIOTR WRÓBLEWSKI,,,False,<class 'datetime.date'>,"New post (Bitcoin price holds $20K, but analys...",None,cromot
6,Bazerka,"Bearpark, England",➡️$zcx https://t.co/NH20Lr02fR ⬅️\n\n➡️$htr ht...,False,<class 'datetime.date'>,"In a bear market, you research! @LeaksBlockcha...",None,Twitter for Android
7,NorthernMiner,,I save my wealth in #bitcoin and I will contin...,False,<class 'datetime.date'>,#Bitcoin always makes a comeback. https://t.co...,[Bitcoin],Twitter for iPhone
8,Bitcoin Vader Official 🏷,Mustafar - Bitcoin beach,Living life with hope !\nafter being ruined by...,False,<class 'datetime.date'>,@maxkeiser @nayibbukele @PeterSchiff OMG that ...,[Bitcoin],Twitter for Android
9,BetFast.com,,Providing real-time wagering information on up...,False,<class 'datetime.date'>,T-Wolves' Gobert eager 'to win a championship'...,"[Bitcoin, Sportsbook]",dlvr.it


In [19]:

tweets_df['date'] = pd.to_datetime(tweets_df['date']).dt.tz_convert(None)
tweets_df


TypeError: <class 'type'> is not convertible to datetime

In [8]:
bitcoin_sentiments = []
for index, row in tweets_df.iterrows():
    try:
        text = row["text"]
        date = row["date"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        passtext = row["text"]
        date = row["date"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
        })
        
    except AttributeError:
        pass

In [104]:
bitcoin_sentiments

[{'text': 'The people who think that #bitcoin is just digital gold or a store-of-value are as wrong as the people who think it’s just a payments system or a medium of exchange.\n\nBitcoin is money: earn, save, spend.',
  'date': datetime.date,
  'compound': 0.0258,
  'positive': 0.084,
  'negative': 0.081,
  'neutral': 0.836},
 {'text': '@Bluebit_io @FoxBusiness Bluesparrow season is coming soon\nbluesparrowtoken is the blue #gem💎💎 in your wallet\n#ETH\n#BlueSparrow\n#BlueSparrowtoken\n#BlueSparrowCommunity\n#BestDrawToken\n#Bluesparrowcharity\n@BlueSparrowETH\n#web3\n#Bitcoin\n@BluesparrowC JYFFJ',
  'date': datetime.date,
  'compound': 0.0,
  'positive': 0.0,
  'negative': 0.0,
  'neutral': 1.0},
 {'text': 'if #bitcoin drops below this blue line, there is trouble straight to sub 20k. it is expected to bounce off it from now on.\nNFA\n @GalaxyTrading_ #crytonews #cryptocurrencies #cryptocurrency https://t.co/rpVJFUZpOo',
  'date': datetime.date,
  'compound': -0.2023,
  'positive': 0.

In [105]:
bitcoin_tweets_df = pd.DataFrame(bitcoin_sentiments, columns=[ "text", "date", "compound", "positive", "negative", "neutral"])
bitcoin_tweets_df

,text,date,compound,positive,negative,neutral
0,The people who think that #bitcoin is just dig...,<class 'datetime.date'>,0.0258,0.084,0.081,0.836
1,@Bluebit_io @FoxBusiness Bluesparrow season is...,<class 'datetime.date'>,0.0000,0.000,0.000,1.000
2,"if #bitcoin drops below this blue line, there ...",<class 'datetime.date'>,-0.2023,0.058,0.083,0.859
3,US inflation data will be 'messy' — 5 things t...,<class 'datetime.date'>,-0.7145,0.000,0.157,0.843
4,@Bluebit_io @FoxBusiness Linking parallel worl...,<class 'datetime.date'>,0.0000,0.000,0.000,1.000
5,"@smdiehl True, but this explanation by profess...",<class 'datetime.date'>,0.2263,0.119,0.000,0.881
6,@joekelly100 @progressivebtcr @RussellRukin @l...,<class 'datetime.date'>,0.6428,0.125,0.061,0.814
7,Texas could not handle those fans spinning. #b...,<class 'datetime.date'>,0.0000,0.000,0.000,1.000
8,@Bluebit_io @FoxBusiness Bluesparrow the futu...,<class 'datetime.date'>,0.0000,0.000,0.000,1.000
9,#Bitcoin Last Price $20621 #BTC 🚀\nDaily Indic...,<class 'datetime.date'>,0.0000,0.000,0.000,1.000
